https://blog.langchain.dev/tutorial-chatgpt-over-your-data/

In [1]:
!wget https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/html-single/about/index -O ocp-index.html

--2024-01-19 10:57:37--  https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/html-single/about/index
Resolving access.redhat.com (access.redhat.com)... 23.33.40.139, 23.33.40.146, 2600:1408:20::6870:eb6b, ...
Connecting to access.redhat.com (access.redhat.com)|23.33.40.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ocp-index.html’

ocp-index.html          [  <=>               ] 143.13K   368KB/s    in 0.4s    

2024-01-19 10:57:38 (368 KB/s) - ‘ocp-index.html’ saved [146569]



In [21]:
!pip install -Uqq langchain langchain-community langchain-core unstructured

In [22]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

loader = UnstructuredHTMLLoader("./ocp-index.html")
data = loader.load()

In [23]:
data

[Document(page_content='Jump To\n\nClose\n\nTable of contents\n\nAbout\n\n1. OpenShift Container Platform 4.14 Documentation\n  \n                    \n        \n          Expand section "1. OpenShift Container Platform 4.14 Documentation"\n        \n        \n          Collapse section "1. OpenShift Container Platform 4.14 Documentation"\n        \n      \n\n      \n                  \n                        \n\n    \n    1.1. Cluster installer activities\n  \n  \n          \n                  \n                        \n\n    \n    1.2. Other cluster installer activities\n  \n  \n          \n                  \n                        \n\n    \n    1.3. Developer activities\n  \n  \n          \n                  \n                          \n\n    \n    1.4. Cluster administrator activities\n  \n                    \n        \n          Expand section "1.4. Cluster administrator activities"\n        \n        \n          Collapse section "1.4. Cluster administrator activities"\n    

In [24]:
data[0].page_content

'Jump To\n\nClose\n\nTable of contents\n\nAbout\n\n1. OpenShift Container Platform 4.14 Documentation\n  \n                    \n        \n          Expand section "1. OpenShift Container Platform 4.14 Documentation"\n        \n        \n          Collapse section "1. OpenShift Container Platform 4.14 Documentation"\n        \n      \n\n      \n                  \n                        \n\n    \n    1.1. Cluster installer activities\n  \n  \n          \n                  \n                        \n\n    \n    1.2. Other cluster installer activities\n  \n  \n          \n                  \n                        \n\n    \n    1.3. Developer activities\n  \n  \n          \n                  \n                          \n\n    \n    1.4. Cluster administrator activities\n  \n                    \n        \n          Expand section "1.4. Cluster administrator activities"\n        \n        \n          Collapse section "1.4. Cluster administrator activities"\n        \n      \n\n      \

In [25]:
from langchain.text_splitter import HTMLHeaderTextSplitter

# https://python.langchain.com/docs/modules/data_connection/document_transformers/HTML_header_metadata

headers_to_split_on = [
    ("h1", "h1"),
    ("h2", "h2"),
    ("h3", "h3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
documents = html_splitter.split_text_from_file("./ocp-index.html")
len(documents)

46

In [26]:
documents

[Document(page_content='Skip to navigation Skip to main content Utilities Infrastructure and Management Cloud Computing Storage Runtimes Integration and Automation'),
 Document(page_content='Subscriptions Downloads Containers Support Cases', metadata={'h3': 'Utilities'}),
 Document(page_content='Subscriptions Downloads Containers Support Cases Products & Services Tools Security Community  \nProducts  \nSupport Services'),
 Document(page_content='Production Support Development Support Product Life Cycles', metadata={'h3': 'Support'}),
 Document(page_content='Consulting Technical Account Management Training & Certifications', metadata={'h3': 'Services'}),
 Document(page_content='Documentation All Documentation Ecosystem Catalog'),
 Document(page_content='Red Hat Enterprise Linux Red Hat JBoss Enterprise Application Platform Red Hat OpenStack Platform Red Hat OpenShift Container Platform', metadata={'h3': 'Documentation'}),
 Document(page_content='Red Hat Partner Ecosystem Partner Resourc

This document is better chunked by list item (li) but this is a header specific splitter.

How about as [PDF](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)?

In [27]:
!wget https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/pdf/about/openshift_container_platform-4.14-about-en-us.pdf -O ocp-about.pdf

--2024-01-19 10:59:16--  https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/pdf/about/openshift_container_platform-4.14-about-en-us.pdf
Resolving access.redhat.com (access.redhat.com)... 23.33.40.139, 23.33.40.146, 2600:1408:20::6870:eb51, ...
Connecting to access.redhat.com (access.redhat.com)|23.33.40.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘ocp-about.pdf’

ocp-about.pdf           [ <=>                ] 135.71K  --.-KB/s    in 0.1s    

2024-01-19 10:59:17 (1.34 MB/s) - ‘ocp-about.pdf’ saved [138966]



In [28]:
!pip install -Uqq pypdf

In [29]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./ocp-about.pdf")
pages = loader.load_and_split()

In [30]:
len(pages)

13

In [31]:
pages

[Document(page_content='OpenShift Container Platform\n \n4.14\nAbout\nIntroduction to OpenShift Container Platform\nLast Updated: 2024-01-17', metadata={'source': './ocp-about.pdf', 'page': 0}),
 Document(page_content='OpenShift Container Platform\n \n4.14\n \nAbout\nIntroduction to OpenShift Container Platform', metadata={'source': './ocp-about.pdf', 'page': 2}),
 Document(page_content='Legal Notice\nCopyright \n©\n 2024 Red Hat, Inc.\nThe text of and illustrations in this document are licensed by Red Hat under a Creative Commons\nAttribution–Share Alike 3.0 Unported license ("CC-BY-SA"). An explanation of CC-BY-SA is\navailable at\nhttp://creativecommons.org/licenses/by-sa/3.0/\n. In accordance with CC-BY-SA, if you distribute this document or an adaptation of it, you must\nprovide the URL for the original version.\nRed Hat, as the licensor of this document, waives the right to enforce, and agrees not to assert,\nSection 4d of CC-BY-SA to the fullest extent permitted by applicable la

This looks better split but still could be better.

In [13]:
?PyPDFLoader

Init signature:
PyPDFLoader(
    file_path: str,
    password: Union[str, bytes, NoneType] = None,
    headers: Optional[Dict] = None,
    extract_images: bool = False,
) -> None
Docstring:     
Load PDF using pypdf into list of documents.

Loader chunks by page and stores page numbers in metadata.
Init docstring: Initialize with a file path.
File:           ~/notebooks/venv/lib64/python3.9/site-packages/langchain_community/document_loaders/pdf.py
Type:           ABCMeta
Subclasses:     

In [14]:
?loader.load_and_split

Signature: loader.load_and_split(text_splitter: 'Optional[TextSplitter]' = None) -> 'List[Document]'
Docstring:
Load Documents and split into chunks. Chunks are returned as Documents.

Args:
    text_splitter: TextSplitter instance to use for splitting documents.
      Defaults to RecursiveCharacterTextSplitter.

Returns:
    List of Documents.
File:      ~/notebooks/venv/lib64/python3.9/site-packages/langchain_community/document_loaders/base.py
Type:      method

In [15]:
!pip uninstall pdfminer pdfminer.six -y

Found existing installation: pdfminer.six 20231228
Uninstalling pdfminer.six-20231228:
  Successfully uninstalled pdfminer.six-20231228


In [18]:
!pip install -Uqq pdf2image pdfminer.six
!pip install -Uqq 'pdfminer.six[image]'
!pip install -Uqq cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [19]:
from langchain_community.document_loaders import UnstructuredPDFLoader
import pdfminer.high_level
from io import StringIO


loader = UnstructuredPDFLoader("./ocp-about.pdf", mode="elements")
data = loader.load()
data

ModuleNotFoundError: No module named 'cv2'

Can't get that to work.  Trying other things.

In [20]:
from langchain_community.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/2302.03803.pdf")
data - loader.load()
data

ModuleNotFoundError: No module named 'cv2'

Still no dice.  Back to the PDF loader.. 

In [ ]:
?PyPDFLoader

https://api.python.langchain.com/en/latest/_modules/langchain_community/document_loaders/pdf.html#
```
headers: Headers to use for GET request to download a file from a web path.
```

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./ocp-about.pdf")
pages = loader.load_and_split()

In [34]:
len(pages)

13

In [35]:
pages

[Document(page_content='OpenShift Container Platform\n \n4.14\nAbout\nIntroduction to OpenShift Container Platform\nLast Updated: 2024-01-17', metadata={'source': './ocp-about.pdf', 'page': 0}),
 Document(page_content='OpenShift Container Platform\n \n4.14\n \nAbout\nIntroduction to OpenShift Container Platform', metadata={'source': './ocp-about.pdf', 'page': 2}),
 Document(page_content='Legal Notice\nCopyright \n©\n 2024 Red Hat, Inc.\nThe text of and illustrations in this document are licensed by Red Hat under a Creative Commons\nAttribution–Share Alike 3.0 Unported license ("CC-BY-SA"). An explanation of CC-BY-SA is\navailable at\nhttp://creativecommons.org/licenses/by-sa/3.0/\n. In accordance with CC-BY-SA, if you distribute this document or an adaptation of it, you must\nprovide the URL for the original version.\nRed Hat, as the licensor of this document, waives the right to enforce, and agrees not to assert,\nSection 4d of CC-BY-SA to the fullest extent permitted by applicable la

In [39]:
!pip install -Uqq langchain-openai faiss-cpu tiktoken

In [42]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

quota.. 

ok, hugging face w/ langchain examples

https://github.com/AndreasFischer1985/code-snippets/blob/master/py/LangChain_HuggingFace_examples.py

In [4]:
!pip install -Uqq sentence-transformers

In [5]:
# https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.huggingface.HuggingFaceEmbeddings.html

from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]